In [1]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import collections
import time

In [3]:
df1 = pd.read_csv('scraping/aku_all_utf.csv', header=None)
df2 = pd.read_csv('scraping/mizu_all_utf.csv', header=None)
aku_data = df1.values
mizu_data = df2.values
# print(aku_data.shape[0])

aku_name = []
mizu_name = []

for i in range(aku_data.shape[0]):
    for j in [0, 2, 4, 6, 8, 10]:
        if aku_data[i, j] != 'ウーラオス悪':
            aku_name.append(aku_data[i, j])

for i in range(mizu_data.shape[0]):
    for j in [0, 2, 4, 6, 8, 10]:
        if mizu_data[i, j] != 'ウーラオス水':
            mizu_name.append(mizu_data[i, j])


print(len(aku_name))  # 3180
print(len(mizu_name))  # 2435

name = np.hstack([aku_name, mizu_name])
print(name.shape)
c = collections.Counter(name)
print(c)

3180
2435
(5615,)
Counter({'サンダー電': 453, 'ミミッキュ': 293, 'ポリゴン２': 251, 'エースバーン': 205, 'ランドロス霊': 173, 'パッチラゴン': 157, 'カバルドン': 156, 'ナットレイ': 145, 'ドラパルト': 138, 'カプ・レヒレ': 124, 'カイリュー': 107, 'アシレーヌ': 99, 'ホルード': 91, 'ゴリランダー': 85, 'ウオノラゴン': 84, 'ザシアン': 83, 'ドリュウズ': 81, 'ウツロイド': 79, 'トゲキッス': 76, 'ドサイドン': 75, 'ジバコイル': 72, 'ドヒドイデ': 67, 'ゲンガー': 66, 'ピクシー': 60, 'バドレックス黒': 58, 'リザードン': 58, 'テッカグヤ': 57, 'ラプラス': 57, 'ヒードラン': 54, 'ラグラージ': 53, 'モロバレル': 53, 'ロトム炎': 52, 'マリルリ': 51, 'ガブリアス': 50, 'レジエレキ': 49, 'ガマゲロゲ': 48, 'キュウコン氷妖': 48, 'ラッキー': 45, 'オーロンゲ': 45, 'ポリゴンＺ': 45, 'バンギラス': 44, 'ニンフィア': 43, 'ウインディ': 38, 'カビゴン': 38, 'ヒヒダルマ氷': 37, 'マンムー': 36, 'カイオーガ': 36, 'ヌオー': 35, 'ムゲンダイナ': 33, 'カミツルギ': 33, 'ブリザポス': 33, 'ギルガルド': 31, 'ギャラドス': 31, 'ハピナス': 30, 'ガオガエン': 28, 'ボルトロス霊': 27, 'ウルガモス': 26, 'アーゴヨン': 26, 'ハッサム': 24, 'メタモン': 24, 'ゴチルゼル': 22, 'カプ・コケコ': 22, 'オンバーン': 22, 'アーマーガア': 21, 'ネクロズマ日': 20, 'エルフーン': 20, 'パッチルドン': 20, 'クレセリア': 19, 'ロトム水': 18, 'ヤドキング毒超': 17, 'エアームド': 17, 'コータス': 16, 'サニーゴ霊': 16, 'マタドガス毒妖': 

In [4]:
print(len(set(name)))  # 240
name_list = list(set(name))  # 辞書からリストへ
name_list = np.sort(name_list)

char_int = {n: i for i, n in enumerate(name_list)}  # 文字から数値
# print(char_int)
int_char = np.array(name_list)  # 数値から文字

aku_int = [char_int[s] for s in aku_name]
mizu_int = [char_int[s] for s in mizu_name]
# print(aku_int)

aku_x = np.zeros((int(np.ceil(len(aku_name)/5)), len(set(name))))
for i in range(len(aku_int)):
    aku_x[i//5, aku_int[i]] = 1.0
aku_y = np.zeros((int(np.ceil(len(aku_name)/5)), 1))
print(aku_x.shape)

mizu_x = np.zeros((int(np.ceil(len(mizu_name)/5)), len(set(name))))
for i in range(len(mizu_int)):
    mizu_x[i//5, mizu_int[i]] = 1.0
mizu_y = np.ones((int(np.ceil(len(mizu_name)/5)), 1))

240
(636, 240)


In [5]:
np.random.seed(0)
np.random.shuffle(aku_x)
np.random.shuffle(mizu_x)
########
train_per = 0.8  # 訓練データの割合
########
aku_num = int(len(aku_name)/5*train_per)
mizu_num = int(len(mizu_name)/5*train_per)
X_train = np.vstack([aku_x[:aku_num], mizu_x[:mizu_num]])
y_train = np.vstack([aku_y[:aku_num], mizu_y[:mizu_num]])
X_test = np.vstack([aku_x[aku_num:], mizu_x[mizu_num:]])
y_test = np.vstack([aku_y[aku_num:], mizu_y[mizu_num:]])
X_all = np.vstack([aku_x, mizu_x])
y_all = np.vstack([aku_y, mizu_y])
print(X_train.shape)
print(y_train.shape)
y_train = np.squeeze(y_train)  # 1次元に落とす場合
y_test = np.squeeze(y_test)
y_all = np.squeeze(y_all)

(897, 240)
(897, 1)


In [29]:
########
Stand = 0  # 正規化
Stand_foward = 0  # テストデータも正規化しちゃう。使わない。
Logic = 0  # ロジスティクス。使わない。
Grid_log = 1  # グリットサーチでロジスティクス
Grid_svc = 0  # グリットサーチでサポートベクターマシン
########

In [7]:
if Stand_foward == 1:
    num_train = np.sum(X_train, axis=0)  # trainに出てくる要素の数

    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    # print(X_test[0])
    for i in range(X_test.shape[0]):
        for j in range(X_test.shape[1]):
            if X_test[i, j] > 10:
                if c[int_char[j]] < 10:
                    print(X_test[i, j], int_char[j], num_train[j])
                    # trainの出現回数が6回以下だと標準偏差が10以上となり、1回だと29.9にもなる。逆に0回だと0になる。
                # X_test[i, j] = 0.0

    X_all = sc.fit_transform(X_all)  # 0をマイナスにしてしまうから、ない方がいい。
    print('正規化しました')  # 全部0を入れたとき、53%ぐらいで悪になる。

In [8]:
if Logic == 1:  # ケンタロス...Logic、standあり58.0%で水、なし55.5%で水
    # 「fit_intercept=False」は切片0
    # 'lbfgs'は、L1正則化ではまだできない。代わりに'liblinear'を使う
    # lr = LogisticRegression(penalty='l1', C=1,
    #                         solver='liblinear', fit_intercept=False)
    lr = LogisticRegression(penalty='l2', C=1, fit_intercept=False)
    # lr = SGDClassifier(loss='log')
    # lr = SGDClassifier(loss='hinge')  # 「ヒツジ損失」ってやつ。SVMと同じ？

    lr.fit(X_train, y_train)

    # print('切片w_0:', lr.intercept_)
    # print('係数w_i:', lr.coef_)
    # L1の時は疎な解である0の係数を含むものもある
    # L2でも0が出るのは訓練データに入ってないデータもあるから
    print('正解率:%.1f%%' % (lr.score(X_test, y_test)*100))

    # lr.fit(X_all, y_all)  # 全部のデータで訓練し直し

In [30]:
if Grid_log == 1:
    param_range = [10**((i-4)/2) for i in range(9)]

    if Stand == 0:  # l1にしたら、消えてしまうマイナーが出てくる。l2にすべき。
        pipe = LogisticRegression(random_state=0,
                                  solver='liblinear', fit_intercept=False)
        # param_grid = [{'C': param_range, 'penalty': ['l1', 'l2']}]
        param_grid = [{'C': [10], 'penalty': ['l2']}]
        # Cが大きいほど、%がはっきりでる。1は結果にパットしない。10は大胆すぎでは？

    if Stand == 1:
        pipe = make_pipeline(StandardScaler(),
                             LogisticRegression(solver='liblinear', fit_intercept=False))
        param_grid = [{'logisticregression__C': [0.38717],
                       'logisticregression__penalty': ['l1']}]
        # ないも入れない場合53.3%で悪になる。
        # l1でC=0.38717のとき、なにも入れない場合50.0%になる。
        # l2にするならマイナーを弱くするためにCは0.01ぐらい。

    gs = GridSearchCV(estimator=pipe, param_grid=param_grid,
                      scoring='accuracy', cv=10)
    gs = gs.fit(X_all, y_all)
    # print('ベストスコア: %.1f%%' % (gs.best_score_*100))
    print('パラメータ:', gs.best_params_)

    lr = gs.best_estimator_
    # print('切片w_0:', lr.intercept_)
    scores = cross_val_score(estimator=lr, X=X_all, y=y_all, cv=10)
    print('訓練時のスコア: %.1f +/- %.1f%%' %
          (np.mean(scores)*100, np.std(scores)*100))

    lr.fit(X_all, y_all)  # 全部のデータで訓練し直し

パラメータ: {'C': 10, 'penalty': 'l2'}
訓練時のスコア: 70.4 +/- 3.5%


In [31]:
if Grid_svc == 1:
    param_range = [10**(i-1) for i in range(3)]

    if Stand == 0:  # マイナーの正解率が下がる
        pipe = SVC(random_state=1)
        param_grid = [{'C': param_range, 'kernel': ['linear']},
                      {'C': param_range, 'gamma': param_range, 'kernel': ['rbf']}]
        # メモ：{'C': 10, 'gamma': 0.1, 'kernel': 'rbf'} 70.9 +/- 5.1%

    if Stand == 1:  # たった3つのグリットで78秒かかってたけど、今スムーズ
        pipe = make_pipeline(StandardScaler(),
                             SVC(random_state=1, kernel='rbf'))
        # param_grid = [{'svc__C': param_range, 'svc__kernel': ['linear']},
        #               {'svc__C': param_range, 'svc__gamma': param_range, 'svc__kernel': ['rbf']}]
        param_grid = [{'svc__C': param_range, 'svc__gamma': param_range}]
        # {'svc__C': 10, 'svc__gamma': 0.1} 60.5 +/- 2.9%
        # {'svc__C': 1, 'svc__gamma': 0.1} 60.8 +/- 3.0%

    gs = GridSearchCV(estimator=pipe, param_grid=param_grid,
                      scoring='accuracy', cv=10)
    gs = gs.fit(X_all, y_all)
    # print('ベストスコア: %.1f%%' % (gs.best_score_*100))
    print('パラメータ:', gs.best_params_)

    lr = gs.best_estimator_
    # print('切片w_0:', lr.intercept_)
    scores = cross_val_score(estimator=lr, X=X_all, y=y_all, cv=10)
    print('訓練時のスコア: %.1f +/- %.1f%%' %
          (np.mean(scores)*100, np.std(scores)*100))

In [32]:
# 実際に検証
start = time.time()

# パーティに入っているポケモン0体以上
all_name = [[''],  # 0体
            ['サンダー電'],
            ['ザシアン'],
            ['サンダー電', 'ザシアン'],
            ['バドレックス黒', 'サンダー電'],
            ['サンダー電', 'ザシアン', 'カバルドン'],
            ['エースバーン', 'サンダー電', 'カバルドン', 'カプ・レヒレ', 'ドラパルト'],
            ['サンダー電', 'ザシアン', 'カバルドン', 'イベルタル', 'ランドロス霊'],]
            
for self_name in all_name:
    # self_int = [char_int[s] for s in self_name]
    self_int = [char_int.get(s) for s in self_name]
    # print(self_int)

    self_x = np.zeros((int(np.ceil(len(self_name)/5)), len(set(name))))
    for i in range(len(self_int)):
        if self_int[i] is None:
            continue
        self_x[i//5, self_int[i]] = 1.0

    aaa = ['悪', '水']
    if Grid_svc == 0:
        pred = lr.predict_proba(self_x)
        pred = np.squeeze(pred)
        print(self_name, ' ウーラオス...%.1f%%で%s' %
              (pred[np.argmax(pred)]*100, aaa[np.argmax(pred)]))

    if Grid_svc == 1:
        print(self_name, 'ウーラオス...', aaa[np.int(lr.predict(self_x))])


# print('かかった時間 %.1f[s]' % (time.time()-start))

['']  ウーラオス...50.0%で悪
['サンダー電']  ウーラオス...60.8%で水
['ザシアン']  ウーラオス...91.6%で悪
['サンダー電', 'ザシアン']  ウーラオス...87.6%で悪
['バドレックス黒', 'サンダー電']  ウーラオス...54.2%で水
['サンダー電', 'ザシアン', 'カバルドン']  ウーラオス...94.1%で悪
['エースバーン', 'サンダー電', 'カバルドン', 'カプ・レヒレ', 'ドラパルト']  ウーラオス...86.9%で悪
['サンダー電', 'ザシアン', 'カバルドン', 'イベルタル', 'ランドロス霊']  ウーラオス...90.4%で悪
